# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

Шаги для выбора локации:

В избранном регионе ищут месторождения, для каждого определяют значения признаков;
Строят модель и оценивают объём запасов;
Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
Прибыль равна суммарной прибыли отобранных месторождений.

# Описание проекта



Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.
Шаги для выбора локации обычно такие:
В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
Строят модель для предсказания объёма запасов в новых скважинах;
Выбирают скважины с самыми высокими оценками значений;
Определяют регион с максимальной суммарной прибылью отобранных скважин.
Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

**Данные геологоразведки трёх регионов находятся в файлах**:
/datasets/geo_data_0.csv. 

/datasets/geo_data_1.csv. 

/datasets/geo_data_2.csv. 


*id* — уникальный идентификатор скважины;

**f0, f1, f2** — три признака точек (неважно, что они означают, но сами признаки значимы);

product — объём запасов в скважине (тыс. баррелей).

*Условия задачи*:

Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

Импортирум все необходимые для работы библиотеки

In [35]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import scipy.stats as st

Прочитаем все три датасета

In [36]:
geodata_0 = pd.read_csv('/datasets/geo_data_0.csv')
geodata_1 = pd.read_csv('/datasets/geo_data_1.csv')
geodata_2 = pd.read_csv('/datasets/geo_data_2.csv')

Выведем общую информацию по всем таблицам

In [37]:
geodata_0.info()
geodata_1.info()
geodata_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8

Видим, что в таблице представлены все значения, нет пустых ячеек.

Удалим во всех датасетах столбец "id" как ненужный для анализа

In [38]:
geodata_0 = geodata_0.drop(['id'],axis = 1)
geodata_1 = geodata_1.drop(['id'],axis = 1)
geodata_2 = geodata_2.drop(['id'],axis = 1)
geodata_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4)
memory usage: 3.1 MB


### Регион *f0*

Целевой признак - столбец *product*, а *f0,f1,f2* - наши признаки.

In [39]:
features = geodata_0.drop(['product'],axis = 1)
target = geodata_0['product']
features1 = geodata_1.drop(['product'],axis = 1)
target1 = geodata_1['product']
features2 = geodata_2.drop(['product'],axis = 1)
target2 = geodata_2['product']

Разобъём выборку на тестовую и валидационную в соотношении 3 к 1, обучим модель и сделаем предсказания с помощью Линейной Регрессии.

In [40]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

In [41]:
linear_model =  LinearRegression()
linear_model.fit(features_train,target_train) 
predicted_valid = linear_model.predict(features_valid) 

linear_result = linear_model.score(features_valid, target_valid)
print("Значение коэффициента детерминации  на валидационной выборке:", linear_result)

Значение коэффициента детерминации  на валидационной выборке: 0.27994321524487786


Посчитаем квадратный корень из среднеквадратичной ошибки *RMSE*

In [42]:
mse = mean_squared_error(target_valid,predicted_valid)
print("RMSE =", mse ** 0.5)

RMSE = 37.5794217150813


Посчитаем средний запас

In [43]:
mean = target.mean()
print(mean)

92.50000000000001


### Регион f1


In [44]:
features1_train, features1_valid, target1_train, target1_valid = train_test_split(features1, target1, test_size=0.25, random_state=12345)
linear_model1 =  LinearRegression()
linear_model1.fit(features1_train,target1_train) 
predicted_valid1 = linear_model1.predict(features1_valid) 

linear_result1 = linear_model1.score(features1_valid, target1_valid)
print("Значение коэффициента детерминации  на валидационной выборке:", linear_result1)

Значение коэффициента детерминации  на валидационной выборке: 0.9996233978805126


In [45]:
mse1 = mean_squared_error(target1_valid,predicted_valid1)
print("RMSE =", mse1 ** 0.5)

RMSE = 0.893099286775616


Посчитаем средний запас

In [46]:
mean1 = target1.mean()
print(mean1)

68.82500000000002


### Регион f2

In [47]:
features2_train, features2_valid, target2_train, target2_valid = train_test_split(features2, target2, test_size=0.25, random_state=12345)
linear_model2 =  LinearRegression()
linear_model2.fit(features2_train,target2_train) 
predicted_valid2 = linear_model2.predict(features2_valid) 

linear_result2 = linear_model2.score(features2_valid, target2_valid)
print("Значение коэффициента детерминации линейной регрессии на валидационной выборке:", linear_result2)

Значение коэффициента детерминации линейной регрессии на валидационной выборке: 0.20524758386040443


In [48]:
mse2 = mean_squared_error(target2_valid,predicted_valid2)
print("RMSE =", mse2 ** 0.5)

RMSE = 40.02970873393434


Посчитаем средний запас

In [49]:
mean2 = target2.mean()
print(mean2)

95.00000000000004


#### Проанализируем результаты между скважинами

Видим, что средний запас самый высокий у третьего региона.

## Обучение и проверка модели

Выпишем все переменные, необходимые для рассчётов. 

In [50]:
points_numbers = 500
best_points_numbers = 200
budget = 10000000000
barrel_price = 450000
probability = 0.025

In [51]:
Borehowl_needs = np.ceil(budget/best_points_numbers/barrel_price)
print(Borehowl_needs)

112.0


Cравним со средним количеством тысяч баррелей по каждому региону в одной скважине

In [52]:
Borehowl_needs_count = Borehowl_needs/mean
Borehowl_needs_count1 = Borehowl_needs/mean1
Borehowl_needs_count2 = Borehowl_needs/mean2
print(Borehowl_needs_count)
print(Borehowl_needs_count1)
print(Borehowl_needs_count2)

1.2108108108108107
1.6273156556483832
1.178947368421052


**Вывод** Для безубыточной разработки скважин необхоимо 112 единиц продукта, но средние запасы в каждом из регионов меньше.

## Подготовка к рассчёту прибыли

Напишем функцию для вычисления прибыли по выбранным скважинам и предсказанным моделям

In [53]:
def profit(target_valid,predicted_valid,best_points_numbers):
    target_valid = target_valid.reset_index(drop=True)
    predicted_valid = pd.DataFrame(predicted_valid).reset_index(drop=True)
    predicted_valid_sorted = predicted_valid.sort_values(by = [0],ascending = False)[:best_points_numbers]
    selected = target_valid[predicted_valid_sorted.index][:best_points_numbers]
    profit = barrel_price*selected.sum() - budget
    return profit


**Вывод** : Самый прибыльный регион - первый. Остальные два несильно отстают от первого.

 ## Рассчёт прибыли и рисков

Посчитаем риски с помощью техники Bootstrap

In [54]:
target_valid = target_valid.reset_index(drop = True)
state = np.random.RandomState(12345)
values = []
for i in range(1000):
    target_subsample = pd.Series(target_valid).sample(points_numbers ,replace = True, random_state = state)
    probs_subsample = pd.Series(predicted_valid)[target_subsample.index]
    values.append(profit(target_subsample,probs_subsample,best_points_numbers))
    
values = pd.Series(values)
mean_value = values.mean()
risk_of_loss = (values < 0).mean()
lower_quantile = values.quantile(0.025)/10**9
upper_quantile = values.quantile(0.975)/10**9

print('Средняя прибыль первого региона: {:.2f} млрд рублей'.format(mean_value/10**9))
confidence_interval = (lower_quantile,upper_quantile)
print("95%-ый доверительный интервал:",confidence_interval)
print("Риск убытков",risk_of_loss,'%')

Средняя прибыль первого региона: 0.40 млрд рублей
95%-ый доверительный интервал: (-0.11121554589049526, 0.9097669415534224)
Риск убытков 0.069 %


Проведём аналогичное исследование для остальных регионов.

In [55]:
target1_valid = target1_valid.reset_index(drop = True)
state = np.random.RandomState(12345)
values1 = []
for i in range(1000):
    target1_subsample = pd.Series(target1_valid).sample(points_numbers ,replace = True, random_state = state)
    probs1_subsample = pd.Series(predicted_valid1)[target1_subsample.index]
    values1.append(profit(target1_subsample,probs1_subsample,best_points_numbers))
    
values1= pd.Series(values1)
mean1_value = values1.mean()
risk_of_loss1 = (values1 < 0).mean()
lower_quantile1 = values1.quantile(0.025)/10**9
upper_quantile1 = values1.quantile(0.975)/10**9

print('Средняя прибыль второго региона: {:.2f} млрд рублей'.format(mean1_value/10**9))
confidence_interval1 = (lower_quantile1,upper_quantile1)
print("95%-ый доверительный интервал:",confidence_interval1)
print("Риск убытков",risk_of_loss1,'%')

Средняя прибыль второго региона: 0.46 млрд рублей
95%-ый доверительный интервал: (0.03382050939898549, 0.852289453866036)
Риск убытков 0.015 %


In [56]:
target2_valid = target2_valid.reset_index(drop = True)
state = np.random.RandomState(12345)
values2 = []
for i in range(1000):
    target2_subsample = pd.Series(target2_valid).sample(points_numbers ,replace = True, random_state = state)
    probs2_subsample = pd.Series(predicted_valid2)[target2_subsample.index]
    values2.append(profit(target2_subsample,probs2_subsample,best_points_numbers))
    
values2 = pd.Series(values2)
mean2_value = values2.mean()
risk_of_loss2 = (values2 < 0).mean()
lower_quantile2 = values2.quantile(0.025)/10**9
upper_quantile2 = values2.quantile(0.975)/10**9

print('Средняя прибыль третьего региона: {:.2f} млрд рублей'.format(mean2_value/10**9))
confidence_interval2 = (lower_quantile2,upper_quantile2)
print("95%-ый доверительный интервал:",confidence_interval2)
print("Риск убытков",risk_of_loss2,'%')

Средняя прибыль третьего региона: 0.40 млрд рублей
95%-ый доверительный интервал: (-0.16335041339559925, 0.9503595749237995)
Риск убытков 0.076 %


**Вывод**: Ни один из регионов не оправдает желаний инвестора.

Добавим результаты для сравнения в таблицу

In [57]:
data = [['RMSE', mse**0.5,mse1**0.5,mse2**0.5],
        ['Средний запас',mean,mean1,mean2],
        ['Средняя прибыль, млрд. рублей',mean_value/10**9,mean1_value/10**9,mean2_value/10**9],
        ['Риск убытков, %',risk_of_loss,risk_of_loss1,risk_of_loss2],
       ]
columns = ['Метрика', 'Регион 0', 'Регион 1','Регион 2']
result = pd.DataFrame(data = data, columns = columns)
result

,Метрика,Регион 0,Регион 1,Регион 2
0,RMSE,37.579422,0.893099,40.029709
1,Средний запас,92.500000,68.825000,95.000000
2,"Средняя прибыль, млрд. рублей",0.396165,0.456045,0.404404
3,"Риск убытков, %",0.069000,0.015000,0.076000


**Вывод**: для разработки могут быть использованы одинаково регион 0 и регион 2, но средняя прибыль не покрывает всех вложений.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован